## METRICS

In [ ]:
!pip install rouge
!pip install gensim
!pip3 install wmd
!pip install POT
!pip install bert-score

In [3]:
from rouge import Rouge
from nltk.translate import meteor_score
from nltk.tokenize import word_tokenize
import gensim
from wmd import WMD
import gensim.downloader as api
from nltk.tokenize import word_tokenize
from bert_score import score
import pandas as pd

In [4]:
# Load a pre-trained Word2Vec model
model = api.load('word2vec-google-news-300')

## ROUGE

In [84]:
def calculate_rouge_scores(reference_text, model_response):
    """
    Calculates ROUGE scores between a reference text and a model response.

    Args:
      reference_text: The reference text.
      model_response: The model response.

    Returns:
      A dictionary containing the ROUGE-1, ROUGE-2, and ROUGE-L scores.
    """
    rouge = Rouge(metrics=['rouge-1', 'rouge-2', 'rouge-l'], stats=['f', 'p', 'r'])
    scores = rouge.get_scores(model_response, reference_text, avg=True)
    return scores

## METEOR

In [85]:
def calculate_meteor_score(reference, hypothesis):
    """
    Calculates the METEOR score for a given hypothesis text with respect to a reference text.

    Args:
      reference (str): Reference text.
      hypothesis (str): Machine-generated text.

    Returns:
      float: METEOR score.
    """
    # Tokenization of texts
    reference_tokens = word_tokenize(reference)
    hypothesis_tokens = word_tokenize(hypothesis)

    # Calculation of the METEOR score
    score = meteor_score.meteor_score([reference_tokens], hypothesis_tokens)
    return score

## WMD

In [86]:
def calculate_wmd_distance(doc1, doc2, w2v_model):
    """
    Computes the Word Mover's Distance between two documents using a pre-trained word2vec model.

    Args:
        doc1 (str): First document.
        doc2 (str): Second document.
        w2v_model (gensim.models.KeyedVectors): Pre-trained Word2Vec model.

    Returns:
        float: WMD distance between the documents.
    """
    # Tokenize the documents and filter words that are in the model's vocabulary
    tokens1 = [word for word in word_tokenize(doc1.lower()) if word in w2v_model.key_to_index]
    tokens2 = [word for word in word_tokenize(doc2.lower()) if word in w2v_model.key_to_index]

    # Calculate the Word Mover's Distance (WMD) using the wmdistance function from gensim
    distance = w2v_model.wmdistance(tokens1, tokens2)
    return distance

## BERTScore

In [87]:
def calculate_bert_score(candidates, references, lang='en', model_type=None):
    """
    Calculates the BERTScore between candidate texts and reference texts.

    Args:
        candidates (list of str): List of texts generated by the machine.
        references (list of str): List of reference texts.
        lang (str): Language code for which the embeddings will be used (default is 'en' for English).
        model_type (str): Type of BERT model to be used (e.g., 'bert-base-uncased'). If None, uses the default model for the language.

    Returns:
        tuple: Three lists containing precision, recall, and F1 scores for each candidate-reference pair.
    """
    # If a specific model is not provided, set a default one
    if not model_type:
        model_type = 'bert-base-multilingual-cased' if lang != 'en' else 'bert-base-uncased'

    # Calculate the BERTScore
    P, R, F1 = score(candidates, references, lang=lang, model_type=model_type, verbose=True)

    return P, R, F1

## Ford Ka 2021

### 1. O que é o Ford Assistance?

In [88]:
reference_text = "O programa Ford Assistance foi criado para oferecer ainda mais tranquilidade aos proprietários de veículos Ford. Isso mesmo! Com esse benefício, a Ford fornece total assistência a seus clientes caso o veículo venha a necessitar de assistência 24 horas, em situações de imobilização do veículo (ou seja, quando o veículo não pode ser utilizado), inclusive em caso de acidentes. Dentre as facilidades estão: guincho, reparo no local, serviço de hospedagem, táxi, devolução do veículo reparado (caso seja necessário), chaveiro, serviço de combustível, telefones úteis, além de carro reserva (exceto em casos de acidentes)."
model_response = "O Ford Assistance é um programa criado para oferecer mais tranquilidade aos proprietários de veículos Ford. Ele fornece assistência total aos clientes caso o veículo necessite de assistência 24 horas, em situações de imobilização do veículo (ou seja, quando o veículo não pode ser utilizado), inclusive em caso de acidentes. Entre as facilidades estão: guincho, reparo no local, serviço de hospedagem, táxi, devolução do veículo reparado (caso seja necessário), chaveiro, serviço de combustível, telefones úteis, além de carro reserva."

In [89]:
scores = calculate_rouge_scores(reference_text, model_response)

In [90]:
meteor = calculate_meteor_score(reference_text, model_response)

In [91]:
distance = calculate_wmd_distance(reference_text, model_response, model)

In [92]:
P, R, F1 = calculate_bert_score([model_response], [reference_text])

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 2.31 seconds, 0.43 sentences/sec


In [93]:
data = {
    "Métrica": ["ROUGE-1", "ROUGE-2", "ROUGE-L", "METEOR", "WMD Distance", "BERTScore"],
    "Precision": [f"{scores['rouge-1']['p']:.2f}", f"{scores['rouge-2']['p']:.2f}", f"{scores['rouge-l']['p']:.2f}", None, None, f"{P.item():.2f}"],
    "Recall": [f"{scores['rouge-1']['r']:.2f}", f"{scores['rouge-2']['r']:.2f}", f"{scores['rouge-l']['r']:.2f}", None, None, f"{R.item():.2f}"],
    "F1": [f"{scores['rouge-1']['f']:.2f}", f"{scores['rouge-2']['f']:.2f}", f"{scores['rouge-l']['f']:.2f}", None, None, f"{F1.item():.2f}"],
    "Score": [None, None, None, f"{meteor:.2f}", f"{distance:.2f}", None]
}

df = pd.DataFrame(data)
df

,Métrica,Precision,Recall,F1,Score
0,ROUGE-1,0.92,0.79,0.85,None
1,ROUGE-2,0.79,0.66,0.72,None
2,ROUGE-L,0.89,0.76,0.82,None
3,METEOR,None,None,None,0.76
4,WMD Distance,None,None,None,0.18
5,BERTScore,0.92,0.89,0.90,None


### 2. Como usar a câmera de Ré?

In [94]:
reference_text = "O sistema de câmera de ré mostrará o que está atrás do veículo quando você deslocar a transmissão para a posição marcha à ré (R). Nota: O sistema de detecção de marcha à ré não é eficaz em velocidades acima de 8 km/h e pode não detectar determinados objetos angulares ou em movimento. Nota: Se a transmissão estiver na posição de marcha à ré (R) e a porta do compartimento de bagagem estiver aberta, nenhuma linha de orientação da câmera de ré será exibida. Nota: Durante o reboque, a câmera vê somente o que está sendo rebocado. Em alguns veículos, as orientações poderão desaparecer quando você ligar o conector do reboque. O sistema pode não funcionar corretamente em uma destas condições: À noite ou em áreas escuras, se as luzes de ré não estiverem funcionando. Se a câmera estiver obstruída. Limpe a lente com um pano macio e sem fiapos e com limpador não abrasivo. Se a câmera estiver desalinhada devido a danos na parte traseira do veículo. O sistema utiliza dois tipos de linhas de orientação para ajudá-lo a ver o que está atrás do veículo: A linha central ajuda a alinhar o centro do veículo a um reboque. As linhas de orientação mostram o caminho real do veículo ao dar marcha ré em uma linha reta. Isso pode ser útil ao dar ré em uma vaga de estacionamento ou alinhar seu veículo a outro objeto."
model_response = "O sistema de câmera de ré mostrará o que está atrás do veículo quando você deslocar a transmissão para a posição marcha à ré (R). Durante a operação, linhas são exibidas na tela representando a trajetória do veículo e a proximidade em relação aos objetos atrás. Quando nota um objeto, o sistema de detecção em marcha à ré exibe destaques em vermelho, amarelo e verde na parte superior da imagem. Você pode ajustar as configurações da câmera usando o menu de configurações do veículo na tela de informações. Para ligar e desligar os auxílios avançados de estacionamento, selecione a opção de configurações na barra de recursos, selecione Veículo, depois selecione Configurações da câmera e ligue e desligue os auxílios avançados de estacionamento. Nota: O sistema de detecção de marcha à ré não é eficaz em velocidades acima de 8 km/h e pode não detectar determinados objetos angulares ou em movimento."

In [95]:
scores = calculate_rouge_scores(reference_text, model_response)

In [96]:
meteor = calculate_meteor_score(reference_text, model_response)

In [97]:
distance = calculate_wmd_distance(reference_text, model_response, model)

In [98]:
P, R, F1 = calculate_bert_score([model_response], [reference_text])

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 2.99 seconds, 0.33 sentences/sec


In [99]:
data = {
    "Métrica": ["ROUGE-1", "ROUGE-2", "ROUGE-L", "METEOR", "WMD Distance", "BERTScore"],
    "Precision": [f"{scores['rouge-1']['p']:.2f}", f"{scores['rouge-2']['p']:.2f}", f"{scores['rouge-l']['p']:.2f}", None, None, f"{P.item():.2f}"],
    "Recall": [f"{scores['rouge-1']['r']:.2f}", f"{scores['rouge-2']['r']:.2f}", f"{scores['rouge-l']['r']:.2f}", None, None, f"{R.item():.2f}"],
    "F1": [f"{scores['rouge-1']['f']:.2f}", f"{scores['rouge-2']['f']:.2f}", f"{scores['rouge-l']['f']:.2f}", None, None, f"{F1.item():.2f}"],
    "Score": [None, None, None, f"{meteor:.2f}", f"{distance:.2f}", None]
}

df = pd.DataFrame(data)
df

,Métrica,Precision,Recall,F1,Score
0,ROUGE-1,0.55,0.39,0.46,None
1,ROUGE-2,0.38,0.26,0.31,None
2,ROUGE-L,0.54,0.38,0.45,None
3,METEOR,None,None,None,0.32
4,WMD Distance,None,None,None,0.35
5,BERTScore,0.80,0.76,0.78,None


### 3. Como desligar a função antiesmagamento?

In [100]:
reference_text = "Para veículos com função anti-esmagamento, proceda conforme indicado para desabilitar esta função de proteção quando houver resistência, por exemplo, no inverno: 1. Feche a janela duas vezes até ela alcançar o ponto de resistência em um curto período e a deixe retroceder. 2. Feche a janela uma terceira vez até o ponto de resistência. Isso desativa a função antiesmagamento e você pode fechar o vidro manualmente. Agora a janela pode passar do ponto de resistência e você pode fechá-la totalmente. Diminua o tempo do procedimento de anti-esmagamento e expressa de um-toque são ativadas novamente. Nota: Entre em contato com uma Concessionária Ford assim que possível se o vidro não fechar após a realização desse procedimento."
model_response = "Para desabilitar a função antiesmagamento, você deve seguir os seguintes passos: 1. Feche a janela duas vezes até ela alcançar o ponto de resistência em um curto período e a deixe retroceder. 2. Feche a janela uma terceira vez até o ponto de resistência. Isso desativa a função antiesmagamento e você pode fechar o vidro manualmente. Porém, tenha cuidado ao fechar a janela para evitar ferimentos ou danos ao veículo. Além disso, a função antiesmagamento será desativada até você redefinir a função. Se o vidro não fechar após a realização desse procedimento, entre em contato com uma Concessionária Ford assim que possível."

In [101]:
scores = calculate_rouge_scores(reference_text, model_response)

In [102]:
meteor = calculate_meteor_score(reference_text, model_response)

In [103]:
distance = calculate_wmd_distance(reference_text, model_response, model)

In [104]:
P, R, F1 = calculate_bert_score([model_response], [reference_text])

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.42 seconds, 0.71 sentences/sec


In [105]:
data = {
    "Métrica": ["ROUGE-1", "ROUGE-2", "ROUGE-L", "METEOR", "WMD Distance", "BERTScore"],
    "Precision": [f"{scores['rouge-1']['p']:.2f}", f"{scores['rouge-2']['p']:.2f}", f"{scores['rouge-l']['p']:.2f}", None, None, f"{P.item():.2f}"],
    "Recall": [f"{scores['rouge-1']['r']:.2f}", f"{scores['rouge-2']['r']:.2f}", f"{scores['rouge-l']['r']:.2f}", None, None, f"{R.item():.2f}"],
    "F1": [f"{scores['rouge-1']['f']:.2f}", f"{scores['rouge-2']['f']:.2f}", f"{scores['rouge-l']['f']:.2f}", None, None, f"{F1.item():.2f}"],
    "Score": [None, None, None, f"{meteor:.2f}", f"{distance:.2f}", None]
}

df = pd.DataFrame(data)
df

,Métrica,Precision,Recall,F1,Score
0,ROUGE-1,0.67,0.60,0.64,None
1,ROUGE-2,0.59,0.51,0.55,None
2,ROUGE-L,0.61,0.55,0.58,None
3,METEOR,None,None,None,0.63
4,WMD Distance,None,None,None,0.35
5,BERTScore,0.88,0.85,0.86,None


## VW Taos 2023

### 1. O que é o controle automático de distância?

In [106]:
reference_text = "O controle automático de distância (ACC = Adaptive Cruise Control) mantém constante uma velocidade ajustada pelo condutor. Se o veículo se aproximar de um veículo à frente, o ACC ajustará a velocidade automaticamente e através disso mantém a distância ajustada por eles."
model_response = "O controle automático de distância, também conhecido como ACC (Adaptive Cruise Control), é um sistema que mantém constante uma velocidade ajustada pelo condutor. Se o veículo se aproximar de um veículo à frente, o ACC ajustará a velocidade automaticamente para manter a distância ajustada."

In [107]:
scores = calculate_rouge_scores(reference_text, model_response)

In [108]:
meteor = calculate_meteor_score(reference_text, model_response)

In [109]:
distance = calculate_wmd_distance(reference_text, model_response, model)

In [110]:
P, R, F1 = calculate_bert_score([model_response], [reference_text])

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.69 seconds, 1.45 sentences/sec


In [111]:
data = {
    "Métrica": ["ROUGE-1", "ROUGE-2", "ROUGE-L", "METEOR", "WMD Distance", "BERTScore"],
    "Precision": [f"{scores['rouge-1']['p']:.2f}", f"{scores['rouge-2']['p']:.2f}", f"{scores['rouge-l']['p']:.2f}", None, None, f"{P.item():.2f}"],
    "Recall": [f"{scores['rouge-1']['r']:.2f}", f"{scores['rouge-2']['r']:.2f}", f"{scores['rouge-l']['r']:.2f}", None, None, f"{R.item():.2f}"],
    "F1": [f"{scores['rouge-1']['f']:.2f}", f"{scores['rouge-2']['f']:.2f}", f"{scores['rouge-l']['f']:.2f}", None, None, f"{F1.item():.2f}"],
    "Score": [None, None, None, f"{meteor:.2f}", f"{distance:.2f}", None]
}

df = pd.DataFrame(data)
df

,Métrica,Precision,Recall,F1,Score
0,ROUGE-1,0.69,0.74,0.71,None
1,ROUGE-2,0.63,0.66,0.64,None
2,ROUGE-L,0.69,0.74,0.71,None
3,METEOR,None,None,None,0.84
4,WMD Distance,None,None,None,0.24
5,BERTScore,0.90,0.91,0.91,None


### 2. O que ocorre no caso de uma intervenção de frenagem automática?

In [112]:
reference_text = "No caso de um obstáculo, o veículo desacelera até parar e é mantido parado por cerca de 2 segundos. 1. Parar o veículo com o pedal de freio após a frenagem. 2. Verificar os arredores."
model_response = "No caso de uma intervenção de frenagem automática, se um obstáculo for detectado, o veículo desacelera até parar e é mantido parado por cerca de 2 segundos. Depois disso, é recomendado que o condutor pare o veículo com o pedal de freio após a frenagem e verifique os arredores."

In [113]:
scores = calculate_rouge_scores(reference_text, model_response)

In [114]:
meteor = calculate_meteor_score(reference_text, model_response)

In [115]:
distance = calculate_wmd_distance(reference_text, model_response, model)

In [116]:
P, R, F1 = calculate_bert_score([model_response], [reference_text])

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 2.41 seconds, 0.41 sentences/sec


In [117]:
data = {
    "Métrica": ["ROUGE-1", "ROUGE-2", "ROUGE-L", "METEOR", "WMD Distance", "BERTScore"],
    "Precision": [f"{scores['rouge-1']['p']:.2f}", f"{scores['rouge-2']['p']:.2f}", f"{scores['rouge-l']['p']:.2f}", None, None, f"{P.item():.2f}"],
    "Recall": [f"{scores['rouge-1']['r']:.2f}", f"{scores['rouge-2']['r']:.2f}", f"{scores['rouge-l']['r']:.2f}", None, None, f"{R.item():.2f}"],
    "F1": [f"{scores['rouge-1']['f']:.2f}", f"{scores['rouge-2']['f']:.2f}", f"{scores['rouge-l']['f']:.2f}", None, None, f"{F1.item():.2f}"],
    "Score": [None, None, None, f"{meteor:.2f}", f"{distance:.2f}", None]
}

df = pd.DataFrame(data)
df

,Métrica,Precision,Recall,F1,Score
0,ROUGE-1,0.64,0.86,0.74,None
1,ROUGE-2,0.51,0.73,0.60,None
2,ROUGE-L,0.64,0.86,0.74,None
3,METEOR,None,None,None,0.77
4,WMD Distance,None,None,None,0.26
5,BERTScore,0.85,0.91,0.88,None


### 3. Qual o óleo recomendado para o motor?

In [118]:
reference_text = "Se nenhum óleo do motor estiver disponível de acordo com a norma prescrita, você poderá completar, em caso de emergência, com um máximo de 0,5 l (0,5 qt) das seguintes normas divergentes uma vez até a próxima troca de óleo regular: — Motores a gasolina: norma VW 504 00 e ACEA C3 ou API SP, viscosidade 0 W-30. — Motores multicombustível: norma VW 502 00 e ACEA A3/B4, viscosidade 5 W-40."
model_response = "O óleo recomendado para o motor depende do tipo de motor. Para motores a gasolina, a norma é VW 504 00 e ACEA C3 ou API SP, com viscosidade 0 W-30. Para motores multicombustível, a norma é VW 502 00 e ACEA A3/B4, com viscosidade 5 W-40. A Volkswagen recomenda o uso de óleo original Volkswagen."

In [119]:
scores = calculate_rouge_scores(reference_text, model_response)

In [120]:
meteor = calculate_meteor_score(reference_text, model_response)

In [121]:
distance = calculate_wmd_distance(reference_text, model_response, model)

In [122]:
P, R, F1 = calculate_bert_score([model_response], [reference_text])

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.36 seconds, 0.73 sentences/sec


In [123]:
data = {
    "Métrica": ["ROUGE-1", "ROUGE-2", "ROUGE-L", "METEOR", "WMD Distance", "BERTScore"],
    "Precision": [f"{scores['rouge-1']['p']:.2f}", f"{scores['rouge-2']['p']:.2f}", f"{scores['rouge-l']['p']:.2f}", None, None, f"{P.item():.2f}"],
    "Recall": [f"{scores['rouge-1']['r']:.2f}", f"{scores['rouge-2']['r']:.2f}", f"{scores['rouge-l']['r']:.2f}", None, None, f"{R.item():.2f}"],
    "F1": [f"{scores['rouge-1']['f']:.2f}", f"{scores['rouge-2']['f']:.2f}", f"{scores['rouge-l']['f']:.2f}", None, None, f"{F1.item():.2f}"],
    "Score": [None, None, None, f"{meteor:.2f}", f"{distance:.2f}", None]
}

df = pd.DataFrame(data)
df

,Métrica,Precision,Recall,F1,Score
0,ROUGE-1,0.59,0.42,0.49,None
1,ROUGE-2,0.35,0.26,0.30,None
2,ROUGE-L,0.54,0.38,0.45,None
3,METEOR,None,None,None,0.50
4,WMD Distance,None,None,None,0.49
5,BERTScore,0.80,0.73,0.77,None
